<a href="https://colab.research.google.com/github/pravinpardeshi/LLM_FineTUning/blob/main/FT_with_Go-Getter_v0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unsloth trl transformers datasets torch

In [ ]:
from unsloth import  FastLanguageModel
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments
from datasets import Dataset
import torch


In [ ]:
# 1. Load the model & tokenizer (CPU mode)
print('Loading')
model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name="unsloth/Llama-3.2-1B-bnb-4bit",  # unquantized CPU-friendly model
    model_name = 'unsloth/Meta-Llama-3.1-8B',
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit=True,
    device_map = 'auto'
)


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    lora_alpha=16,
    lora_dropout=0, #0.05,
    bias="none",
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    max_seq_length = 2048,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


In [ ]:
data = [
        {"instruction": "What is the Go-Getter book about?", "input": "", "response": "It is about determination and perseverance"},
        {"instruction": "Write an alternative ending where The Blue Vase fails the test.", "input": "", "output": "In this version, Cappy decides not to pursue the blue vase, leading Cappy Ricks to reconsider his judgment about who truly deserves to lead."},

    # Add more samples here...
]

dataset = Dataset.from_list(data)



In [ ]:
def formatting_func(example):
    return {
        "text": f"""### Instruction:
        {example['instruction']}

        ### Input:
        {example['input']}

        ### Response:
        {example['response']}"""
    }

dataset = dataset.map(formatting_func)


In [ ]:
training_args = SFTConfig(
        dataset_text_field = "text",
        output_dir="./sft_finetune_cpu",
        max_seq_length = 2048,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        logging_steps = 1,
        optim = "adamw_8bit",
        fp16=False, # no fp16 on CPU
        seed = 3407,
)



In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=training_args,
    formatting_func=formatting_func,
)



In [ ]:
print('Trainng the model')
print(f"Model is loaded on: {next(model.parameters()).device}")
trainer.train()
